In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
import visdom
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

In [3]:
class CommonBlock(nn.Module):
    def __init__(self, in_channel, out_channel, stride=1):        # A regular Block can simply complete two convolution operations
        super(CommonBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)

    def forward(self, x):
        identity = x                                            #The shortcut for a regular Block is a direct connection and does not require dimensional upscaling or downsampling

        x = F.relu(self.bn1(self.conv1(x)), inplace=True)       # Complete a convolution
        x = self.bn2(self.conv2(x))                             # The second convolution does not add the relu activation function

        x += identity                                           # Sum of two paths
        return F.relu(x, inplace=True)                          # Add an activation function output

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")                         # 创建GPU运算环境
print(device)

class SpecialBlock(nn.Module):
    def __init__(self, in_channel, out_channel, stride):
        super(SpecialBlock, self).__init__()
        self.change_channel = nn.Sequential(
            nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channel)
        )
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)

    def forward(self, x):
        identity = self.change_channel(x)
        out = F.relu(self.bn1(self.conv1(x)), inplace=True)
        out = self.bn2(self.conv2(out))
        out += identity
        return F.relu(out, inplace=True)
                    # Output convolutional unit

class ResNet18(nn.Module):
    def __init__(self, classes_num):
        super(ResNet18, self).__init__()
        self.prepare = nn.Sequential(           # The common preprocessing for all Resnets == [batch, 64, 56, 56]
            nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.layer1 = nn.Sequential(            # layer1 is a bit special. Since both the input and output channels are 64, there are two common blocks
            CommonBlock(64, 64, stride=1),
            CommonBlock(64, 64, stride=1)
        )
        self.layer2 = nn.Sequential(            # layer234 is similar. Due to the different input and output channels, there is one SpecialBlock and one CommonBlock
            SpecialBlock(64, 128, stride=2),
            CommonBlock(128, 128, stride=1)
        )
        self.layer3 = nn.Sequential(
            SpecialBlock(128, 256, stride=2),
            CommonBlock(256, 256, stride=1)
        )
        self.layer4 = nn.Sequential(
            SpecialBlock(256, 512, stride=2),
            CommonBlock(512, 512, stride=1)
        )
        self.pool = nn.AdaptiveAvgPool2d(output_size=(1, 1))    # Convolution is completed through an adaptive mean pooling == [batch, 512, 1, 1]
        self.fc = nn.Sequential(                # The fully connected layer finally used for classification varies flexibly as needed
            nn.Dropout(p=0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(256, classes_num)
        )

    def forward(self, x):
        x = self.prepare(x)         # prepare

        x = self.layer1(x)          # 4 conv units
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.pool(x)            # max pooling
        x = x.flatten(1)   # flatten x
        x = self.fc(x)
        return x

cpu


In [5]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd
import os
BATCH_SIZE = 32


# ==== path ====
img_dir = 'Dataset Binary'                  # picture file
label_file = 'Dataset_Labels.xlsx'          # label file

img_dir = 'Dataset Binary'
train_csv = 'spine_train_split.csv'
test_csv  = 'spine_test_split.csv'
save_path = "./Spine_ResNet18.pth"
# ==== read label ====


df = pd.read_excel(label_file)
df.columns = ['Spine_Name', 'Spine_Label']

# To facilitate indexing, we convert the DataFrame into a dict
label_dict = dict(zip(df['Spine_Name'], df['Spine_Label']))

# ==== data enforce ====


class BinarySpineDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.data = dataframe.reset_index(drop=True)  # The received DataFrame no longer reads the csv
        self.root_dir = root_dir
        self.transform = transform

        self.label_map = {
            "Mushroom": 0,
            "Stubby": 1,
            "Thin": 2
        }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        img_path = os.path.join(self.root_dir, row['Spine_Name'])
        image = Image.open(img_path).convert("RGB")

        label_str = row['Spine_Label']
        label = self.label_map[label_str]   # ← Here convert the string to 0/1/2

        if self.transform:
            image = self.transform(image)

        return image, label

# ==== Divide the training set and the test set ====
train_df = pd.read_csv(train_csv)
test_df  = pd.read_csv(test_csv)

data_transform = {
    "train": transforms.Compose([
        transforms.Resize((250, 250)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
    ]),
    "val": transforms.Compose([
        transforms.Resize((250, 250)),
        transforms.ToTensor(),
    ])
}

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

train_dataset = BinarySpineDataset(train_df, img_dir, transform=data_transform["train"])
val_dataset   = BinarySpineDataset(val_df,   img_dir, transform=data_transform["val"])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False)

In [6]:
import torch
import visdom
from PIL import Image
model = ResNet18(classes_num=3)
from torch.nn import CrossEntropyLoss
from torch import optim

BATCH_SIZE = 32
EPOCH = 30
save_path = "./Spine_ResNet18.pth"    # The location for saving model weight parameters

In [9]:
# Create a visdom to visualize the training and testing situations
viz = visdom.Visdom(env="spine_exp")

# Test functions, input models and data reading interfaces
def evalute(model, loader):
    correct = 0
    total = len(loader.dataset)
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        # The validation and test processes do not require backpropagation
        model.eval()
        with torch.no_grad():
            out = model(x)
            prediction = out.argmax(dim=1)
        correct += torch.eq(prediction, y).float().sum().item()
    return correct / total


net = ResNet18(classes_num=3)
#net.load_state_dict(torch.load(save_path))          # Continue training using the weights from the last training
optimizer = optim.Adam(net.parameters(), lr=0.0001)
loss_function = CrossEntropyLoss()                  # The cross-entropy loss function is used in multi-classification problems

best_acc = 0.0
best_epoch = 0
global_step = 0
EPOCH = 30

for epoch in range(EPOCH):
    net.train()
    running_loss = 0.0

    for step, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # update loss line
        viz.line([loss.item()], [global_step], win='resnet_loss', update='append')
        global_step += 1

        rate = (step + 1) / len(train_loader)
        a = "*" * int(rate * 50)
        b = "." * int((1 - rate) * 50)
        print(f"\repoch: {epoch+1} train loss: {int(rate*100):3d}% [{a}->{b}] {loss:.4f}", end="")

    # ===== Perform a validation once per epoch =====
    test_acc = evalute(net, test_loader)
    print(f"  epoch {epoch+1} test acc: {test_acc:.4f}")

    # ===== keep best model =====
    if test_acc > best_acc:
        best_acc = test_acc
        best_epoch = epoch + 1

        torch.save({
            'epoch': best_epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_acc': best_acc
        }, save_path)

        print(f"\n🔥 Saved best model at epoch {best_epoch}, acc={best_acc:.4f}")

print("Finish !")
print("Best epoch:", best_epoch, " Best Acc:", best_acc)


Setting up a new session...


epoch: 1 train loss:  25% [************->.....................................] 1.0377

KeyboardInterrupt: 